# Temporal-Difference Methods

In this notebook, you will write your own implementations of many Temporal-Difference (TD) methods.

While we have provided some starter code, you are welcome to erase these hints and write your code from scratch.

---

### Part 0: Explore CliffWalkingEnv

We begin by importing the necessary packages.

In [ ]:
import sys
import gym
import numpy as np
from collections import defaultdict, deque
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

import check_test
from plot_utils import plot_values

Use the code cell below to create an instance of the [CliffWalking](https://github.com/openai/gym/blob/master/gym/envs/toy_text/cliffwalking.py) environment.

In [ ]:
env = gym.make('CliffWalking-v0')

The agent moves through a $4\times 12$ gridworld, with states numbered as follows:
```
[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
 [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]
```
At the start of any episode, state `36` is the initial state.  State `47` is the only terminal state, and the cliff corresponds to states `37` through `46`.

The agent has 4 potential actions:
```
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
```

Thus, $\mathcal{S}^+=\{0, 1, \ldots, 47\}$, and $\mathcal{A} =\{0, 1, 2, 3\}$.  Verify this by running the code cell below.

In [ ]:
print(env.action_space)
print(env.observation_space)

In this mini-project, we will build towards finding the optimal policy for the CliffWalking environment.  The optimal state-value function is visualized below.  Please take the time now to make sure that you understand _why_ this is the optimal state-value function.

_**Note**: You can safely ignore the values of the cliff "states" as these are not true states from which the agent can make decisions.  For the cliff "states", the state-value function is not well-defined._

In [ ]:
# define the optimal state-value function
V_opt = np.zeros((4,12))
V_opt[0:13][0] = -np.arange(3, 15)[::-1]
V_opt[0:13][1] = -np.arange(3, 15)[::-1] + 1
V_opt[0:13][2] = -np.arange(3, 15)[::-1] + 2
V_opt[3][0] = -13

plot_values(V_opt)

### Part 1: TD Control: Sarsa

In this section, you will write your own implementation of the Sarsa control algorithm.

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

Please complete the function in the code cell below.

(_Feel free to define additional functions to help you to organize your code._)

In [ ]:
def generate_episode_from_policy(env, policy, epsilon=0.1, seed=None):
    episode = []
    state = env.reset(seed = seed)[0]
    while True:
        action = policy[state]
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        next_state, reward, terminated, truncated, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if terminated or truncated:
            break
    return episode

def generate_episode_from_Q(env, Q, epsilon=0.1, seed=None):
    policy = {k: np.argmax(v) for k, v in Q.items()}
    return generate_episode_from_policy(env, policy, epsilon, seed)

def epsilon_greedy(env, Qstate, epsilon):
    if np.random.rand() < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Qstate)
    return action

def sarsa0_estimating_function(Q, state, action, next_state, gamma, epsilon):
    next_action = epsilon_greedy(env, Q[next_state], epsilon)
    return Q[next_state][next_action]

def sarsamax_estimating_function(Q, state, action, next_state, gamma, epsilon):
    next_action = epsilon_greedy(env, Q[next_state], 0)
    return Q[next_state][next_action]

def expected_sarsa_estimating_function(Q, state, action, next_state, gamma, epsilon):
    optimal_action = epsilon_greedy(env, Q[next_state], 0)
    probs = np.ones(env.nA)*epsilon/(env.nA)
    probs[optimal_action] += 1-epsilon
    return np.dot(Q[next_state],probs)

test = check_test.Tests()

def generic_sarsa(env, num_episodes, alpha, estimating_function, gamma=1.0):
    # initialize action-value function (empty dictionary of arrays)
    Q = np.zeros((env.nS, env.nA))
    # initialize performance monitor
    # loop over episodes
    bar = tqdm(range(1, num_episodes+1))
    for i_episode in bar:
        # monitor progress
        state = env.reset()[0]
        # epsilon = max((1-1e-4)**i_episode, 0.1)
        # epsilon = 0.1*(1-5e-5)**i_episode
        epsilon = 0.001
        sum_reward = 0
        while True:
            action = epsilon_greedy(env, Q[state], epsilon)
            next_state, reward, terminated, truncated, info = env.step(action)
            sum_reward += reward
            if terminated or truncated:
                Q[state][action] += alpha*(reward - Q[state][action])
                break
            Gt = estimating_function(Q, state, action, next_state, gamma, epsilon)
            Q[state][action] += alpha*(reward + gamma*Gt - Q[state][action])
            state = next_state
        if i_episode % 1000 == 0:
            np.set_printoptions(precision=1)
            bar.set_postfix({
                "Epsilon": epsilon,
                             "Reward": Q[36][0],
                             "Risky path": str(np.argmax(Q[24:34,:], axis=1)),
                             })
        ## TODO: complete the function

    return Q

# estimating_function = sarsa0_estimating_function
estimating_function = sarsamax_estimating_function
# estimating_function = expected_sarsa_estimating_function

# obtain the estimated optimal policy and corresponding action-value function
Q_sarsa = generic_sarsa(env, 5000, .01, estimating_function)
# print the estimated optimal policy
policy_sarsa = np.argmax(Q_sarsa,axis=1).reshape(4,12)
check_test.run_check('td_control_check', policy_sarsa)
print("\nEstimated Optimal Policy (UP = 0, RIGHT = 1, DOWN = 2, LEFT = 3, N/A = -1):")
print(policy_sarsa)

# plot the estimated optimal state-value function
V_sarsa = ([np.max(Q_sarsa[key]) for key in np.arange(48)])
plot_values(V_sarsa)

Use the next code cell to visualize the **_estimated_** optimal policy and the corresponding state-value function.  

If the code cell returns **PASSED**, then you have implemented the function correctly!  Feel free to change the `num_episodes` and `alpha` parameters that are supplied to the function.  However, if you'd like to ensure the accuracy of the unit test, please do not change the value of `gamma` from the default.